In [3]:
!pip install requests beautifulsoup4 pandas


  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)

   ---------------------------------------- 0/7 [urllib3]
   ----------- ---------------------------- 2/7 [idna]
   ----------------- ---------------------- 3/7 [charset_normalizer]
   ---------------------------- ----------- 5/7 [requests]
   ---------------------------------------- 7/7 [beautifulsoup4]



In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Fct  de scrapping

#TODO : to be updated

def scrape_reviews(url, num_pages=1):
    all_reviews = []
    for page in range(1, num_pages + 1):
        page_url = f"{url}?page={page}" # Adapter l'URL de pagination
        print(f"Scraping page: {page_url}")
        try:
            response = requests.get(page_url, headers={'User-Agent': 'Mozilla/5.0'})
            response.raise_for_status() # Lève une exception pour les codes d'état HTTP d'erreur
            soup = BeautifulSoup(response.text, 'html.parser')

            # Exemple de sélecteurs - À ADAPTER ABSOLUMENT à la structure du site cible
            review_containers = soup.find_all('div', class_='review-item') # Ex: div avec classe 'review-item'

            for container in review_containers:
                review_text = container.find('p', class_='review-text').get_text(strip=True) if container.find('p', class_='review-text') else "N/A"
                rating = container.find('span', class_='rating').get_text(strip=True) if container.find('span', class_='rating') else "N/A"
                # Ajoutez d'autres champs comme date, auteur, etc.

                all_reviews.append({
                    'review_text': review_text,
                    'rating': rating
                })
        except requests.exceptions.RequestException as e:
            print(f"Erreur lors de la requête vers {page_url}: {e}")
        time.sleep(2) # Soyez poli avec le serveur, attendez un peu entre les requêtes

    return pd.DataFrame(all_reviews)

# Exemple d'utilisation (URL à remplacer par celle du site tunisien cible)
# url_cible = "https://www.mytek.tn/smartphone-samsung-galaxy-a54-5g-8-256go.html#reviews"
# df_reviews = scrape_reviews(url_cible, num_pages=5)
# df_reviews.to_csv('raw_customer_reviews.csv', index=False)
# print(df_reviews.head())


#chat version

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_tunisianet_reviews(product_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }

    response = requests.get(product_url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # Bloc contenant les avis
    reviews_section = soup.find("div", id="product_reviews_block")

    if not reviews_section:
        print("❌ Aucun avis trouvé sur cette page.")
        return pd.DataFrame()

    all_reviews = reviews_section.find_all("div", class_="review-item")
    
    data = []

    for review in all_reviews:

        # Nom du client
        name_tag = review.find("span", class_="reviewer-name")
        name = name_tag.text.strip() if name_tag else "Inconnu"

        # Date de l'avis
        date_tag = review.find("span", class_="review-date")
        date = date_tag.text.strip() if date_tag else "Inconnue"

        # Note (nombre d’étoiles)
        star_container = review.find("div", class_="star_content")
        rating = len(star_container.find_all("i", class_="fa fa-star")) if star_container else None

        # Commentaire
        comment_tag = review.find("p", class_="review-text")
        comment = comment_tag.text.strip() if comment_tag else ""

        # Sauvegarde
        data.append({
            "name": name,
            "date": date,
            "rating": rating,
            "comment": comment
        })

    return pd.DataFrame(data)


# -------------------------------
# 🔥 Tester avec le Redmi Note 13
# -------------------------------

#url = "https://www.tunisianet.com.tn/smartphone/71310-smartphone-xiaomi-redmi-note-13-8-go-256-go-noir.html"
url = "https://www.tunisianet.com.tn/smartphone/71347-smartphone-samsung-galaxy-a54-5g-128-go-8-go-ram-lime.html"

df = scrape_tunisianet_reviews(url)

print(df)

df.to_csv("samsung_a54.csv", index=False)
print("✔️ CSV enregistré ")


❌ Aucun avis trouvé sur cette page.
Empty DataFrame
Columns: []
Index: []
✔️ CSV enregistré 


Code de scraping pour Orange Tunisie (HTML statique)

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_orange_reviews(product_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }

    response = requests.get(product_url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # Section des avis
    review_blocks = soup.find_all("div", class_="review")  # à adapter selon le site

    if not review_blocks:
        print("❌ Aucun avis trouvé.")
        return pd.DataFrame()

    data = []

    for review in review_blocks:
        # Nom
        name_tag = review.find("span", class_="reviewer-name")
        name = name_tag.text.strip() if name_tag else "Inconnu"

        # Note
        rating_tag = review.find("div", class_="stars")  # vérifier le HTML exact
        rating = rating_tag["data-rating"] if rating_tag else None

        # Commentaire
        comment_tag = review.find("p", class_="review-text")
        comment = comment_tag.text.strip() if comment_tag else ""

        data.append({
            "name": name,
            "rating": rating,
            "comment": comment
        })

    df = pd.DataFrame(data)
    return df

# -------------------------
# Tester
# -------------------------
url = "https://www.orange.tn/fr/smartphones/samsung-galaxy-a54"
df = scrape_orange_reviews(url)
print(df.head())

df.to_csv("reviews_orange_galaxy_a54.csv", index=False)
print("✅ CSV sauvegardé.")


❌ Aucun avis trouvé.
Empty DataFrame
Columns: []
Index: []
✅ CSV sauvegardé.


# =>
Le problème rencontré avec Beautiful Soup est typique car de nombreux sites, y compris ceux comme Orange et Ooredoo en Tunisie, masquent les avis clients via des techniques dynamiques (JavaScript, chargement asynchrone, données JSON cachées) qui ne sont pas accessibles par un simple parsing HTML.

Solutions pour contourner ce problème
Extraction via données JSON cachées : Certains sites stockent les avis dans des structures JSON invisibles directement dans le code source, qui peuvent être extraites après avoir identifié les bons endpoints ou structures. Par exemple, des sites comme Tripadvisor utilisent cette méthode pour leurs avis clients.

Scraping dynamique avec Selenium ou Puppeteer : Ces outils simulent un navigateur complet qui exécute le JavaScript afin de charger entièrement la page et ses avis avant extraction.

Utilisation d'APIs ou sources alternatives : Parfois, les plateformes ont des APIs publiques ou semi-publiques pour accéder aux avis clients, ou bien on peut extraire les avis depuis des sites tiers comme Trustpilot qui référencent les avis des clients pour ces opérateurs.

Automatisation via plateformes comme Make.com + Apify : Certaines plateformes proposent des modules déjà prêts pour collecter des avis de sites comme Trustpilot, avec des options d'analyse, ce qui pourrait faciliter la collecte.

Exemples spécifiques aux opérateurs en Tunisie
Avis pour Orange Tunisie peuvent être trouvés sur Tripadvisor et Trustpilot mais pas facilement scrapés via BeautifulSoup à cause du contenu dynamique.​

Avis pour Ooredoo Tunisie existent aussi sur Trustpilot et Indeed mais de même, nécessitent des méthodes avancées de scraping ou API.​

Tutoriels sur le scraping d'avis clients montrent que juste BeautifulSoup est souvent insuffisant pour scraper tous les avis car il faut gérer la partie JavaScript ou JSON cachée.​

Pour notre mini-projet on prefere opter pour  : notamment Selenium pour le scraping dynamique, ou bien l'extraction directe de JSON cachés, ou encore d'utiliser des sources comme Trustpilot pour l'analyse des avis clients.

option 1:Selenium

Simule un vrai navigateur

Charge le JS → les avis deviennent visibles

Compatible avec Tunisianet / Orange / Ooredoo

Cette méthode est la plus sûre pour scraper des avis en Tunisie aujourd’hui



Option 2 : Extraire des avis depuis les réseaux sociaux ou forums

Facebook, Twitter, forums de tech tunisiens

Souvent visibles sans JS compliqué

Mais nécessite un peu de nettoyage

I WILL WORK ON BUSINESS HOTEL

MAPS:

https://www.google.com/maps/place/Business+H%C3%B4tel/@36.818325,10.1820211,17z/data=!4m8!3m7!1s0x12fd3489b5a4f4e1:0xdda07f445b5b03cd!8m2!3d36.818325!4d10.184596!9m1!1b1!16s%2Fg%2F11bwn563xh?entry=ttu&g_ep=EgoyMDI1MTEyMy4xIKXMDSoASAFQAw%3D%3D


avis dans site de booking:
https://www.booking.com/reviews/tn/hotel/business.fr.html?label=gen173nr-10CA0o4gFCCGJ1c2luZXNzSDNYBGjiAYgBAZgBM7gBF8gBDNgBA-gBAfgBAYgCAagCAbgCl86gyQbAAgHSAiRlZjRhOGM4ZS1hNzA0LTRkYzItYjIyNC1hNWMxMjk2YWI0YmXYAgHgAgE&sid=32b2429f18b4624fc09d32f51f4441cc&customer_type=total&hp_nav=0&keep_landing=1&order=featuredreviews&page=1&r_lang=fr&rows=75&